# YOLOv11s Fine-tuning for Chest X-ray Abnormality Detection


## Section 1: Setup and Imports

In [1]:
!uv pip install -q roboflow ultralytics wandb tqdm pillow numpy

In [2]:
!uv pip uninstall albumentations

Using Python 3.11.13 environment at: /usr
Uninstalled 1 package in 123ms
 - albumentations==2.0.8


In [3]:
import os
os.environ["ALBUMENTATIONS_DISABLE"] = "1"
# Import required libraries
import shutil
from pathlib import Path
import yaml

import torch
import wandb
from ultralytics import YOLO, settings

# Import custom augmentation
import sys
sys.path.insert(0, str(Path.cwd()))

print("✓ Imports successful")
print(f"  PyTorch version: {torch.__version__}")
print(f"  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Imports successful
  PyTorch version: 2.6.0+cu124
  CUDA available: True
  GPU: Tesla T4
  GPU Memory: 15.8 GB


## Section 2: Verify Preprocessed Data

In [4]:
import gdown
gdown.download(quiet=True, id="1x09wTm0d-bpZvEunga-9q697WK8d8ftN")

'preprocessed_2classes_aug.zip'

In [7]:
os.makedirs('data/', exist_ok=True)
!unzip -q /kaggle/working/preprocessed_2classes_aug.zip -d data/

In [9]:
!cat /kaggle/working/data/preprocessed_2classes_aug/data.yaml

path: /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/preprocessed_2classes_aug
train: train/images
val: valid/images
test: test/images
nc: 2
names:
- Aortic enlargement
- Cardiomegaly


In [10]:
# Correct paths in data.yaml
data_yaml_path = Path('/kaggle/working/data/preprocessed_2classes_aug/data.yaml')

if data_yaml_path.exists():
    print(f"Correcting paths in {data_yaml_path}")
    with open(data_yaml_path, 'r') as f:
        data_yaml_content = f.read()

    # Replace the incorrect path with the correct one
    corrected_yaml_content = data_yaml_content.replace(
        "/home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/preprocessed_2classes_aug",
        "/kaggle/working/data/preprocessed_2classes_aug"
    )

    with open(data_yaml_path, 'w') as f:
        f.write(corrected_yaml_content)

    print("✓ Paths corrected successfully!")
    print("\nContent of corrected data.yaml:")
    print("-" * 80)
    print(corrected_yaml_content)
    print("-" * 80)

else:
    print(f"Error: data.yaml not found at {data_yaml_path}")
    raise FileNotFoundError(f"data.yaml not found at {data_yaml_path}")

Correcting paths in /kaggle/working/data/preprocessed_2classes_aug/data.yaml
✓ Paths corrected successfully!

Content of corrected data.yaml:
--------------------------------------------------------------------------------
path: /kaggle/working/data/preprocessed_2classes_aug
train: train/images
val: valid/images
test: test/images
nc: 2
names:
- Aortic enlargement
- Cardiomegaly

--------------------------------------------------------------------------------


In [11]:
# Verify preprocessed data directory
preprocessed_dir = Path('data/preprocessed_2classes_aug')
data_yaml = preprocessed_dir / 'data.yaml'

print("Verifying Preprocessed Data")
print("=" * 80)

if not preprocessed_dir.exists():
    print("ERROR: Preprocessed data not found!")
    print(f"   Expected location: {preprocessed_dir.absolute()}")
    print("\nPlease run data_preparation.ipynb first to create preprocessed data.")
    raise FileNotFoundError(f"Preprocessed data not found at {preprocessed_dir}")

if not data_yaml.exists():
    print(f"ERROR: data.yaml not found at {data_yaml}")
    raise FileNotFoundError(f"data.yaml not found")

print(f"✓ Preprocessed data directory found: {preprocessed_dir}")
print(f"✓ Data YAML found: {data_yaml}")

# Load data.yaml
with open(data_yaml, 'r') as f:
    data_config = yaml.safe_load(f)

print(f"\nDataset Configuration:")
print(f"  Number of classes: {data_config['nc']}")
print(f"  Class names: {data_config['names']}")

# Count images in each split
splits = ['train', 'valid', 'test']
split_counts = {}

for split in splits:
    images_dir = preprocessed_dir / split / 'images'
    if images_dir.exists():
        count = len(list(images_dir.glob('*.png'))) + len(list(images_dir.glob('*.jpg')))
        split_counts[split] = count
    else:
        split_counts[split] = 0

print(f"\nDataset Statistics:")
print(f"  Train:      {split_counts['train']:,} images")
print(f"  Validation: {split_counts['valid']:,} images")
print(f"  Test:       {split_counts['test']:,} images")
print(f"  Total:      {sum(split_counts.values()):,} images")

if split_counts['train'] == 0:
    print("\nERROR: No training images found!")
    raise ValueError("No training images found in preprocessed data")

print("\n✓ Data verification complete - ready for training!")
print("=" * 80)

Verifying Preprocessed Data
✓ Preprocessed data directory found: data/preprocessed_2classes_aug
✓ Data YAML found: data/preprocessed_2classes_aug/data.yaml

Dataset Configuration:
  Number of classes: 2
  Class names: ['Aortic enlargement', 'Cardiomegaly']

Dataset Statistics:
  Train:      7,086 images
  Validation: 1,056 images
  Test:       492 images
  Total:      8,634 images

✓ Data verification complete - ready for training!


## Section 3: WandB Setup

In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
key = user_secrets.get_secret("wandb_api_key")


In [13]:
# Login to WandB
wandb.login(key=key)
print("✓ Logged into Weights & Biases successfully")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: minhquana (minhquana-university-of-transportation-and-communication) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✓ Logged into Weights & Biases successfully


In [14]:
NAME="yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes"
PROJECT="chest-xray-abnormality-detection"
EPOCH=200
BATCH_SIZE=48
IMG_SIZE=1024
PATIENCE=10
OPTIMIZER="AdamW"
LR=0.00005
LRF=0.001
DEVICE=[0, 1]
WARMUP_EPOCH=10.0
WEIGHT_DECAY=0.001

In [15]:
# Initialize WandB project
wandb.init(
    project=PROJECT,
    name=NAME,
    config={
        "model": "YOLOv11s",
        "dataset": "VinBigData Chest X-ray v3 (Preprocessed + Filtered)",
        "epochs": EPOCH,
        "batch_size": BATCH_SIZE,
        "image_size": IMG_SIZE,
        "patience": PATIENCE,
        "optimizer": OPTIMIZER,
        "learning_rate": LR,
        "preprocessing": "grayscale + histogram_eq + normalization (NO blur)",
        "augmentation": "gaussian_blur (custom callback) + rotation (YOLO degrees=5.0)",
        "training_strategy": "minimal augmentation to preserve medical features",
        "gaussian_blur": "80% 3x3, 20% 5x5 with sigma=0.5",
    }
)

print("✓ WandB initialized successfully")
print(f"  Project: chest-xray-abnormality-detection")
print(f"  Run name: {wandb.run.name}")
print(f"  Run URL: {wandb.run.url}")

✓ WandB initialized successfully
  Project: chest-xray-abnormality-detection
  Run name: yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes
  Run URL: https://wandb.ai/minhquana-university-of-transportation-and-communication/chest-xray-abnormality-detection/runs/l6dko0ic


In [16]:
# Enable WandB integration in Ultralytics
settings.update({'wandb': True})

print("✓ WandB integration enabled for Ultralytics YOLO")
print("\nTraining metrics will be automatically logged to WandB:")
print("   - Loss curves (box_loss, cls_loss, dfl_loss)")
print("   - mAP scores (mAP50, mAP50-95)")
print("   - Learning rate schedules")
print("   - Training/validation images with predictions")

✓ WandB integration enabled for Ultralytics YOLO

Training metrics will be automatically logged to WandB:
   - Loss curves (box_loss, cls_loss, dfl_loss)
   - mAP scores (mAP50, mAP50-95)
   - Learning rate schedules
   - Training/validation images with predictions


## Section 4: Training Configuration

Configure training parameters with minimal augmentation strategy.

In [17]:
# Training configuration
training_config = {
    # Data
    'data': str(data_yaml),
    
    # Training hyperparameters
    'epochs': EPOCH,
    'batch': BATCH_SIZE,
    'imgsz': IMG_SIZE,
    'patience': PATIENCE,
    'save': True,
    'plots': True,
    'verbose': True,
    
    # Device and performance
    'device': DEVICE,
    'workers': 8,
    'cache': False,
    
    # Optimization parameters
    'optimizer': OPTIMIZER,
    'lr0': LR,
    'lrf': LRF,          # Final learning rate (lr0 * lrf)
    'momentum': 0.937,
    'weight_decay': WEIGHT_DECAY,
    'warmup_epochs': WARMUP_EPOCH,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'cos_lr': True,         # Use cosine learning rate scheduler
    
    # Only rotation is enabled from YOLO built-in augmentations
    # 'degrees': 5.0,
    # 'hsv_h': 0.0,    
    # 'hsv_s': 0.0,  
    # 'hsv_v': 0.0,    
    # 'translate': 0.0,   
    # 'scale': 0.0,    
    # 'shear': 0.0,          
    # 'perspective': 0.0,  
    # 'fliplr': 0.0, 
    # 'flipud': 0.0,
    # 'mosaic': 0.0,
    # 'mixup': 0.0,  
    # 'copy_paste': 0.0,    
    # 'auto_augment': None,  
    # 'erasing': 0.0,        
    'degrees': 10.0,         # Tăng độ xoay một chút
    'hsv_h': 0.015,          # Thêm một chút biến đổi màu sắc (dù ảnh xám)
    'hsv_s': 0.7,            # Tăng biến đổi độ bão hòa
    'hsv_v': 0.4,            # QUAN TRỌNG: Thêm biến đổi độ sáng/tương phản
    'translate': 0.1,        # Dịch chuyển ảnh +/- 10%
    'scale': 0.5,            # Co giãn ảnh +/- 50%
    'shear': 2.0,            # Thêm biến đổi cắt xéo
    'perspective': 0.001,    # Thêm một chút biến đổi góc nhìn
    'fliplr': 0.5,           # Lật ngang 50% ảnh. Dù giải phẫu bị đảo, nó giúp mô hình học tổng quát.
    'flipud': 0.0,           # Không nên lật dọc với X-quang ngực
    'mosaic': 1.0,           # Bật Mosaic augmentation
    'mixup': 0.1,            # Bật Mixup với tỷ lệ thấp
    'copy_paste': 0.3,       # Rất hữu ích cho class imbalance, hãy thử bật nó
}

print("Training Configuration")
print("=" * 80)
for key, value in training_config.items():
    print(f"  {key:25s}: {value}")
print("=" * 80)

Training Configuration
  data                     : data/preprocessed_2classes_aug/data.yaml
  epochs                   : 200
  batch                    : 48
  imgsz                    : 1024
  patience                 : 10
  save                     : True
  plots                    : True
  verbose                  : True
  device                   : [0, 1]
  workers                  : 8
  cache                    : False
  optimizer                : AdamW
  lr0                      : 5e-05
  lrf                      : 0.001
  momentum                 : 0.937
  weight_decay             : 0.001
  warmup_epochs            : 10.0
  warmup_momentum          : 0.8
  warmup_bias_lr           : 0.1
  cos_lr                   : True
  degrees                  : 10.0
  hsv_h                    : 0.015
  hsv_s                    : 0.7
  hsv_v                    : 0.4
  translate                : 0.1
  scale                    : 0.5
  shear                    : 2.0
  perspective              : 

## Section 5: Model Training

Train YOLOv11s with custom augmentation callback.

In [18]:
# Load YOLOv11s model
print("\nLoading YOLOv11s model...")
model = YOLO('yolo11s.pt')

print("✓ Model loaded successfully")
print(f"  Model architecture: YOLOv11s")
print(f"  Parameters: ~{sum(p.numel() for p in model.model.parameters()) / 1e6:.1f}M")

print("\nStarting training...")
print("Progress will be tracked in WandB dashboard")
print("-" * 80)


Loading YOLOv11s model...
✓ Model loaded successfully
  Model architecture: YOLOv11s
  Parameters: ~9.5M

Starting training...
Progress will be tracked in WandB dashboard
--------------------------------------------------------------------------------


In [19]:
# Train the model
try:
    results = model.train(
        **training_config,
        project=PROJECT,
        name=NAME
    )
    
    print("\n" + "=" * 80)
    print("✓ Training completed successfully!")
    print("=" * 80)
    
    # Check if results is valid
    if results is None:
        print("\nWARNING: Training returned None - checking training directory...")
        # Find the latest training directory
        training_dir = Path('chest-xray-abnormality-detection')
        if training_dir.exists():
            # Get all run directories sorted by modification time
            run_dirs = sorted(training_dir.glob('yolov11s-gaussian-blur-rotation-Adam*'), 
                            key=lambda x: x.stat().st_mtime, reverse=True)
            if run_dirs:
                latest_run = run_dirs[0]
                print(f"  Found latest training run: {latest_run}")
                best_model_path = latest_run / 'weights' / 'best.pt'
                last_model_path = latest_run / 'weights' / 'last.pt'
                
                if best_model_path.exists():
                    print(f"  ✓ Best model found: {best_model_path}")
                elif last_model_path.exists():
                    print(f"  ✓ Last model found: {last_model_path}")
                    best_model_path = last_model_path
                else:
                    print(f"  No model weights found in {latest_run}")
                    best_model_path = None
            else:
                print("  No training runs found")
                best_model_path = None
        else:
            print("  Training directory not found")
            best_model_path = None
    else:
        # Display results
        print("\nTraining Results:")
        if hasattr(results, 'results_dict'):
            print(f"  Best mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")
            print(f"  Best mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
        
        # Save best model path
        if hasattr(results, 'save_dir') and results.save_dir:
            best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
            print(f"\nBest model saved to: {best_model_path}")
        else:
            print("\nWARNING: results.save_dir not available")
            best_model_path = None
    
except Exception as e:
    print(f"\nTraining failed: {e}")
    import traceback
    traceback.print_exc()
    best_model_path = None

Ultralytics 8.3.227 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/preprocessed_2classes_aug/data.yaml, degrees=10.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=5e-05, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11s-gaussian-blur-rotation-AdamW-lo

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Overriding model.yaml nc=80 with nc=2
Transferred 493/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1857.3±652.9 MB/s, size: 83.8 KB)
train: Scanning /kaggle/working/data/preprocessed_2classes_aug/train/labels... 7086 images, 2362 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 7086/7086 1.7Kit/s 4.3s0.1s
train: New cache created: /kaggle/working/data/preprocessed_2classes_aug/train/labels.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 546.5±274.6 MB/s, size: 88.7 KB)
val: Scanning /kaggle/working/data/preprocessed_2classes_aug/valid/labels... 1056 images, 352 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1056/1056 2.1Kit/s 0.5s1ss
val: New cache created: /kaggle/working/data/preprocessed_2classes_aug/valid/labels.cache
Plotting labels to /kaggle/working/chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1056       1124      0.869       0.89      0.933        0.5
    Aortic enlargement        632        632      0.866      0.862      0.904      0.467
          Cardiomegaly        492        492      0.871      0.919      0.962      0.533
Speed: 0.3ms preprocess, 7.7ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /kaggle/working/chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes


wandb:                                                                                



✓ Training completed successfully!

  Found latest training run: chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes
  ✓ Best model found: chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes/weights/best.pt


## Section 6: Model Validation

In [20]:
# Validate on test set
print("Model Validation on Test Set")
print("=" * 80)

# Load best model
if 'best_model_path' in locals() and best_model_path.exists():
    print(f"Loading best model: {best_model_path}")
    model = YOLO(str(best_model_path))
else:
    print("Using last trained model")

# model = YOLO("/kaggle/working/chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment/weights/best.pt")

print("\nRunning validation...")
metrics = model.val(data=str(data_yaml), split='test')

print("\nValidation Results:")
print("=" * 80)
results_dict = metrics.results_dict
print(f"  mAP50:       {results_dict.get('metrics/mAP50(B)', 0):.4f}")
print(f"  mAP50-95:    {results_dict.get('metrics/mAP50-95(B)', 0):.4f}")
print(f"  Precision:   {results_dict.get('metrics/precision(B)', 0):.4f}")
print(f"  Recall:      {results_dict.get('metrics/recall(B)', 0):.4f}")
print("=" * 80) 

Model Validation on Test Set
Loading best model: chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes/weights/best.pt

Running validation...
Ultralytics 8.3.227 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1810.5±601.9 MB/s, size: 87.2 KB)
val: Scanning /kaggle/working/data/preprocessed_2classes_aug/test/labels... 492 images, 164 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 492/492 1.6Kit/s 0.3s0.0ss
val: New cache created: /kaggle/working/data/preprocessed_2classes_aug/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 31/31 1.8it/s 17.2s0.6s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        492        519      0.856      0.879      0.903      0.485
    Aortic enlargement        301        301      0.861      0.863      0.881      0.471
          Cardiomegaly        218        218      0.852      0.894      0.925      0.498
Speed: 2.6ms preprocess, 28.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /kaggle/working/runs/detect/val

Validation Results:
  mAP50:       0.9027
  mAP50-95:    0.4847
  Precision:   0.8565
  Recall:      0.8789


## Section 7: Model Export

In [21]:
# Export to backend
backend_models_dir = Path('backend/models')
backend_models_dir.mkdir(parents=True, exist_ok=True)

target_model_path = backend_models_dir / 'yolov11s_finetuned.pt'

print("Exporting Model to Backend")
print("=" * 80)

# Check if best_model_path exists
if 'best_model_path' not in locals() or best_model_path is None:
    print("WARNING: best_model_path not found from training")
    print("   Searching for latest trained model...")
    
    # Try to find the latest model
    training_dir = Path('chest-xray-abnormality-detection')
    if training_dir.exists():
        run_dirs = sorted(training_dir.glob('yolov11s-gaussian-blur-rotation-Adam*'), 
                        key=lambda x: x.stat().st_mtime, reverse=True)
        if run_dirs:
            latest_run = run_dirs[0]
            best_model_path = latest_run / 'weights' / 'best.pt'
            if not best_model_path.exists():
                best_model_path = latest_run / 'weights' / 'last.pt'
            
            if best_model_path.exists():
                print(f"   ✓ Found model: {best_model_path}")
            else:
                print(f"   No model weights found")
                best_model_path = None
        else:
            print("   No training runs found")
            best_model_path = None
    else:
        print("   Training directory not found")
        best_model_path = None

if best_model_path and best_model_path.exists():
    print(f"Source: {best_model_path}")
    print(f"Target: {target_model_path}")
    
    shutil.copy(best_model_path, target_model_path)
    
    if target_model_path.exists():
        size_mb = target_model_path.stat().st_size / (1024*1024)
        print(f"\n✓ Model exported successfully!")
        print(f"  File size: {size_mb:.2f} MB")
        print(f"  Location: {target_model_path}")
        print(f"\nModel ready for production use!")
    else:
        print("Export failed")
else:
    print("Cannot export - model not found")
    print("\nPlease check:")
    print("  1. Training completed successfully")
    print("  2. Model weights exist in training directory")
    print("  3. No errors during training")

print("=" * 80)

Exporting Model to Backend
Source: chest-xray-abnormality-detection/yolov11s-gaussian-blur-rotation-AdamW-low-lr-strong-augment-2classes/weights/best.pt
Target: backend/models/yolov11s_finetuned.pt

✓ Model exported successfully!
  File size: 18.37 MB
  Location: backend/models/yolov11s_finetuned.pt

Model ready for production use!


In [22]:
# Close WandB run
wandb.finish()
print("✓ WandB run finished")

✓ WandB run finished


## Section 8: Training Summary

In [23]:
print("\nTRAINING SUMMARY")
print("=" * 80)

print("\nCompleted Tasks:")
print("  1. ✓ Loaded preprocessed data from data/preprocessed/")
print("  2. ✓ Applied custom Gaussian blur augmentation during training")
print("  3. ✓ Applied YOLO rotation augmentation (±5°)")
print("  4. ✓ Trained YOLOv11s model for 100 epochs with early stopping")
print("  5. ✓ Tracked training with WandB")
print("  6. ✓ Validated on test set")
print("  7. ✓ Exported best model to backend/models/")

print("\nFinal Metrics:")
if 'results_dict' in locals():
    print(f"  mAP50:       {results_dict.get('metrics/mAP50(B)', 'N/A')}")
    print(f"  mAP50-95:    {results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
    print(f"  Precision:   {results_dict.get('metrics/precision(B)', 'N/A')}")
    print(f"  Recall:      {results_dict.get('metrics/recall(B)', 'N/A')}")

print("\n" + "=" * 80)


TRAINING SUMMARY

Completed Tasks:
  1. ✓ Loaded preprocessed data from data/preprocessed/
  2. ✓ Applied custom Gaussian blur augmentation during training
  3. ✓ Applied YOLO rotation augmentation (±5°)
  4. ✓ Trained YOLOv11s model for 100 epochs with early stopping
  5. ✓ Tracked training with WandB
  6. ✓ Validated on test set
  7. ✓ Exported best model to backend/models/

Final Metrics:
  mAP50:       0.9026635466848129
  mAP50-95:    0.48471503697716145
  Precision:   0.8564855566137338
  Recall:      0.8789308305952152

